<a href="https://colab.research.google.com/github/shiehn/rune_notebooks/blob/main/elixir_musicgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

INSTALL MUSIC_GEN

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!python -m pip install setuptools wheel
!python -m pip install -U audiocraft
!pip install ffmpeg-python
!pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121


from audiocraft.models import MusicGen
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write
import torchaudio

model = MusicGen.get_pretrained('medium', device='cuda')


!pip install dawnet-client


import os
import soundfile as sf
import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "4cf3dc9b-fb8a-49c0-805c-1920d167ad81" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(audio_description: str = "hiphop",
                      audio_length: int = 8,
                      num_samples: int = 1):
    try:
        model.set_generation_params(duration=audio_length)

        for i in range(num_samples):
          output = model.generate(
              descriptions=[audio_description],
              progress=True, return_tokens=True
          )

          output_audio_name = f"generated_audio_{i}.wav"

          # Move the tensor to CPU, convert to numpy array, reshape and ensure the data type is float32
          generated_audio = output[0].cpu().numpy().astype('float32')
          generated_audio = generated_audio.squeeze()  # Remove extra dimensions
          sf.write(output_audio_name, generated_audio, 32000, subtype='PCM_16')  # Save as WAV file


          # after executing your custom code you send data back to the plugin like so ..
          await dawnet.output().add_file(output_audio_name)


        await dawnet.output().add_message("Returning the input file")
        await dawnet.output().send()

        return True
    except Exception as e:
        print(f"Error in arbitrary_method: {e}")
        return f"Method encountered an error: {e}"


dawnet.set_token(token=dawnet_token)
dawnet.set_name("MusicGen Text-To-Audio")
dawnet.set_description("Provide a text description, number of samples, and sample lenth in seconds")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()

Looking in indexes: https://download.pytorch.org/whl/cu121


/usr/local/lib/python3.10/dist-packages/audiocraft/models/musicgen.py:127: UserWarning: MusicGen pretrained model relying on deprecated checkpoint mapping. Please use full pre-trained id instead: facebook/musicgen-medium
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


ConnectionRefusedError: [Errno 111] Connect call failed ('34.135.228.111', 8765)

See our repo https://github.com/facebookresearch/audiocraft for more details on how to use this model!

See also [MusicGen Gradio Demo](https://colab.research.google.com/drive/1-Xe9NCdIs2sCUbiSmwHXozK6AAhMm7_i?usp=sharing) for a Colab using the Gradio app instead!
